In [2]:
import pandas as pd
import ibis
from ibis import selectors as s 
from ibis import _
ibis.options.interactive=True

data = pd.read_csv('2024-02-02_-_Worker_and_Temporary_Worker.csv')

In [13]:
t=ibis.memtable(data)
t_clean=(
    t
    .mutate(town=_["Town/City"].upper())
    .filter(_.Route == 'Skilled Worker')
)
t_town=(
    t_clean
    .group_by(_.town)
    .aggregate(n=_.count())
    .order_by(_.n.desc())
)

display(t_town.head(10))
t_town.filter(_.town=='EXETER')


┏━━━━━━━━━━━━┳━━━━━━━┓
┃ town       ┃ n     ┃
┡━━━━━━━━━━━━╇━━━━━━━┩
│ string     │ int64 │
├────────────┼───────┤
│ LONDON     │ 24901 │
│ BIRMINGHAM │  1962 │
│ MANCHESTER │  1814 │
│ BRISTOL    │   891 │
│ LEICESTER  │   878 │
│ EDINBURGH  │   748 │
│ LEEDS      │   736 │
│ GLASGOW    │   732 │
│ NOTTINGHAM │   700 │
│ READING    │   664 │
└────────────┴───────┘

┏━━━━━━━━┳━━━━━━━┓
┃ town   ┃ n     ┃
┡━━━━━━━━╇━━━━━━━┩
│ string │ int64 │
├────────┼───────┤
│ EXETER │   124 │
└────────┴───────┘

In [4]:
t_clean.filter(_.town=='EXETER')

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Organisation Name                   ┃ Town/City ┃ County ┃ Type & Rating     ┃ Route          ┃ town   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ string                              │ string    │ string │ string            │ string         │ string │
├─────────────────────────────────────┼───────────┼────────┼───────────────────┼────────────────┼────────┤
│ 3DFORUS Limited                     │ Exeter    │ NULL   │ Worker (A rating) │ Skilled Worker │ EXETER │
│ A J Sands Ltd T/A Sands Consultants │ Exeter    │ Devon  │ Worker (A rating) │ Skilled Worker │ EXETER │
│ A&S 2014 Ltd                        │ Exeter    │ NULL   │ Worker (A rating) │ Skilled Worker │ EXETER │
│ A2Z SPICE LTD                       │ exeter    │ NULL   │ Worker (A rating) │ Skilled Worker │ EXETER │
│ ABBEY 24X7 LIMITED                  │ EXETER    │ NULL   │ Worker (A rating) │ Skilled Worker │ EXETER │
│ Alexander House                     │ Exeter    │ NULL   │ Worker (A rating) │ Skilled Worker │ EXETER │
│ Amelia House Residential House      │ Exeter    │ Devon  │ Worker (A rating) │ Skilled Worker │ EXETER │
│ AnTech Ltd                          │ Exeter    │ Devon  │ Worker (A rating) │ Skilled Worker │ EXETER │
│ Appen (Europe) LTD                  │ Exeter    │ Devon  │ Worker (A rating) │ Skilled Worker │ EXETER │
│ Artigiano Espresso Bars limited     │ Exeter    │ NULL   │ Worker (A rating) │ Skilled Worker │ EXETER │
│ …                                   │ …         │ …      │ …                 │ …              │ …      │
└─────────────────────────────────────┴───────────┴────────┴───────────────────┴────────────────┴────────┘

### Check if Company has Sponsor

In [5]:
t.filter(_['Organisation Name'].contains('Vitrifi'))

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Organisation Name ┃ Town/City ┃ County ┃ Type & Rating     ┃ Route          ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ string            │ string    │ string │ string            │ string         │
├───────────────────┼───────────┼────────┼───────────────────┼────────────────┤
│ Vitrifi Limited   │ London    │ NULL   │ Worker (A rating) │ Skilled Worker │
└───────────────────┴───────────┴────────┴───────────────────┴────────────────┘

### Get a list of Company House Companies

In [2]:
import requests
from requests.auth import HTTPBasicAuth as Auth

# stream_key='ce33ea20-3dd9-4cbe-8a57-1ad0396d7c78'
rest_key = 'a66a1cd0-cf10-4c36-9759-6a4d0b3b8053'
# steam_url='https://stream.companieshouse.gov.uk/companies'
rest_url='https://api.company-information.service.gov.uk/advanced-search/companies'
r = requests.get(
    rest_url,
    auth=(rest_key,'')
    )
assert r.status_code==200
cps = r.json().get('items')

In [7]:
rest_url='https://api.company-information.service.gov.uk/search'
r = requests.get(
    rest_url,
    auth=(rest_key,''),
    params={'q':'Full Fibre'}
    )
assert r.status_code==200
cps = r.json().get('items')

In [17]:
import pandas as pd
import polars as pl
# pl.DataFrame(cps)

In [25]:
# from linkedin import linkedin
CLIENT='78wtdy9ojz691l'
SECRET='HIEr80EKn1HoM2bf'
RETURN_URL = 'http://localhost:8000'

authentication = linkedin.LinkedInAuthentication(
                    CLIENT,
                    SECRET,
                    RETURN_URL,
                    linkedin.PERMISSIONS.enums.values()
                )
authentication.state

In [26]:
TOKEN='AQVV0_b4MMsKlnDnoRERuliSj0lbd_0UyXKQazy7EN3cvzg_DpUYduXVxKh1eYmJC8aYWRoTnJfrFAcwOANVydv2SIF5EiRYCQXWFUEs6NBWCLCE-Oi1mPrDxvAud1U8WKdVDXlSokb1XwVCPv0a_LNDjNZ0sNBucD55DELt5zdLfx53rbKqIwA0eajo4UwxsEJduJHrvvZ7vW-3U_r5Dpo4d2S4cGa6kKQu5d7u64OtYVFyJsDkht3AXg71RcywBbexOXVBU7K2ZfEmp-_z4f2RAp3ivFotNjq-xqGmvj5l4kssPygrzSKCV_nwYBCZWsqSDhkBTZ31xQqtQcdS8iNEkfwcGQ'

In [30]:
from linkedin_api.clients.restli.client import RestliClient

restli_client = RestliClient()

# response = restli_client.get(
#   resource_path="/userinfo",
#   access_token=TOKEN
# )
# print(response.entity)
response = restli_client.finder(
  resource_path="/adAccounts",
  finder_name="search",
  query_params={
    "search": {
      "status": {
        "values": ["ACTIVE", "DRAFT"]
      },
      "reference": {
        "values": ["urn:li:organization:123"]
      },
      "test": True
    }
  },
  version_string="202212",
  access_token=TOKEN
)
ad_accounts = response.elements

### Linkedin Job Scrapper

In [33]:
LI_AT_COOKIE='AQEDASNg818EzqL8AAABjZrgniMAAAGNvu0iI04ARx1E_aPuXAJYfbHnU7fpUmbqTAV195FTdpws7bULRtzcfylM1WQbugwNlOClc4CfVMwHoMeZhKK6nq136vcgR'
# this just has to reset everytime
import os 
os.environ['LI_AT_COOKIE']=LI_AT_COOKIE

In [ ]:
import logging
from linkedin_jobs_scraper import LinkedinScraper
from linkedin_jobs_scraper.events import Events, EventData, EventMetrics
from linkedin_jobs_scraper.query import Query, QueryOptions, QueryFilters
from linkedin_jobs_scraper.filters import RelevanceFilters, TimeFilters, TypeFilters, ExperienceLevelFilters, \
    OnSiteOrRemoteFilters

# Change root logger level (default is WARN)
logging.basicConfig(level=logging.INFO)


# Fired once for each successfully processed job
def on_data(data: EventData):
    print('[ON_DATA]', data.title, data.company, data.company_link, data.date, data.link, data.insights,
          len(data.description))

# Fired once for each page (25 jobs)
def on_metrics(metrics: EventMetrics):
    print('[ON_METRICS]', str(metrics))

def on_error(error):
    print('[ON_ERROR]', error)

def on_end():
    print('[ON_END]')


scraper = LinkedinScraper(
    chrome_executable_path=None,  # Custom Chrome executable path (e.g. /foo/bar/bin/chromedriver)
    chrome_binary_location=None,  # Custom path to Chrome/Chromium binary (e.g. /foo/bar/chrome-mac/Chromium.app/Contents/MacOS/Chromium)
    chrome_options=None,  # Custom Chrome options here
    headless=True,  # Overrides headless mode only if chrome_options is None
    max_workers=1,  # How many threads will be spawned to run queries concurrently (one Chrome driver for each thread)
    slow_mo=0.5,  # Slow down the scraper to avoid 'Too many requests 429' errors (in seconds)
    page_load_timeout=40  # Page load timeout (in seconds)    
)

# Add event listeners
scraper.on(Events.DATA, on_data)
scraper.on(Events.ERROR, on_error)
scraper.on(Events.END, on_end)

queries = [
    Query(
        options=QueryOptions(
            limit=27  # Limit the number of jobs to scrape.            
        )
    ),
    Query(
        query='Data Analyst',
        options=QueryOptions(
            locations=['United Kingdom', 'Europe'],
            apply_link=True,  # Try to extract apply link (easy applies are skipped). If set to True, scraping is slower because an additional page must be navigated. Default to False.
            skip_promoted_jobs=True,  # Skip promoted jobs. Default to False.
            page_offset=2,  # How many pages to skip
            limit=5,
            filters=QueryFilters(
                company_jobs_url='https://www.linkedin.com/jobs/search/?f_C=1441%2C17876832%2C791962%2C2374003%2C18950635%2C16140%2C10440912&geoId=92000000',  # Filter by companies.                
                relevance=RelevanceFilters.RECENT,
                time=TimeFilters.MONTH,
                type=[TypeFilters.FULL_TIME, TypeFilters.INTERNSHIP],
                on_site_or_remote=[OnSiteOrRemoteFilters.REMOTE],
                experience=[ExperienceLevelFilters.MID_SENIOR]
            )
        )
    ),
]

scraper.run(queries)

In [42]:
QueryFilters(
                company_jobs_url='https://www.linkedin.com/jobs/search/?f_C=1441%2C17876832%2C791962%2C2374003%2C18950635%2C16140%2C10440912&geoId=92000000',  # Filter by companies.                
                relevance=RelevanceFilters.RECENT,
                time=TimeFilters.MONTH,
                type=[TypeFilters.FULL_TIME, TypeFilters.INTERNSHIP],
                on_site_or_remote=[OnSiteOrRemoteFilters.REMOTE],
                experience=[ExperienceLevelFilters.MID_SENIOR]
            )